In [1]:
import os
from dotenv import load_dotenv
from typing import List
from datahub.emitter.mcp import MetadataChangeProposalWrapper
import datahub.metadata.schema_classes as models
import datahub.emitter.mce_builder as builder
from datahub.emitter.rest_emitter import DatahubRestEmitter
load_dotenv()

datashub_gms_server = os.getenv('DATAHUB_GMS_SERVER')
datahub_token = os.getenv('DATAHUB_TOKEN')
datahub_actor = os.getenv('DATAHUB_ACTOR')
datahub_env = 'DEV'


In [2]:
def emit_dataset_lineage(dataset: str, upstream_datasets: List[str], emitter: DatahubRestEmitter):
    mcpw = MetadataChangeProposalWrapper(
        "dataset",
        models.ChangeTypeClass.UPSERT,
        entityUrn=dataset,
        aspectName="upstreamLineage",
        aspect=models.UpstreamLineageClass(upstreams=[models.UpstreamClass(dataset=d, type=models.DatasetLineageTypeClass.TRANSFORMED) for d in upstream_datasets])
        )
    emitter.emit_mcp(mcp=mcpw)

emitter = DatahubRestEmitter(
    gms_server=datashub_gms_server, 
    token=datashub_gms_server, 
    extra_headers={'X-DataHub-Actor': datahub_actor}
    )
    
emit_dataset_lineage(
    dataset=builder.make_dataset_urn("bigquery", "tmp.table_b", datahub_env), 
    upstream_datasets=[builder.make_dataset_urn("bigquery", "tmp.table_a", datahub_env)],
    emitter=emitter
)

OperationalError: ('Unable to emit metadata to DataHub GMS', {'message': '401 Client Error: Unauthorized for url: https://netdata.acryl.io/gms/aspects?action=ingestProposal'})